<a href="https://colab.research.google.com/github/soobook/PyTorch-DL/blob/main/code/PT02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2회차: PyTorch 설치 및 기초 코딩

### CUDA(Compute Unified Device Architecture)
- NVIDIA에서 만든 병렬 연산용 프로그래밍 프레임워크
- GPU를 범용 컴퓨팅(GPGPU: General Purpose GPU) 작업에 활용

In [ ]:
import torch

print(f'파이토치 버전: {torch.__version__}')
print("GPU" if torch.cuda.is_available() else "CPU")

파이토치 버전: 2.8.0+cu126
GPU


In [ ]:
# torch 및 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
epochs = 5

In [ ]:
# 데이터 전처리 및 로딩
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 507kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.71MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.72MB/s]


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# train_loader 정보 출력
print("=== Train Loader Info ===")
print(f"총 배치 수 (len): {len(train_loader)}")    # 전체 배치 개수
print(f"총 샘플 수: {len(train_loader.dataset)}")  # 전체 데이터 수
print(f"총 배치 수 (직접계산): {len(train_loader.dataset)//batch_size + 1}")

# 첫 번째 배치 샘플 shape 확인
for images, labels in train_loader:
    print(f"이미지 shape: {images.shape}")         # 예: [batch_size, 1, 28, 28]
    print(f"레이블 shape: {labels.shape}")         # 예: [batch_size]
    print(f"데이터 타입: {images.dtype}, 라벨 dtype: {labels.dtype}")
    break

=== Train Loader Info ===
총 배치 수 (len): 938
총 샘플 수: 60000
총 배치 수 (직접계산): 938
이미지 shape: torch.Size([64, 1, 28, 28])
레이블 shape: torch.Size([64])
데이터 타입: torch.float32, 라벨 dtype: torch.int64


In [ ]:
# test_loader 정보 출력
print("\n=== Test Loader Info ===")
print(f"총 배치 수 (len): {len(test_loader)}")
print(f"총 샘플 수: {len(test_loader.dataset)}")

for images, labels in test_loader:
    print(f"이미지 shape: {images.shape}")
    print(f"레이블 shape: {labels.shape}")
    print(f"데이터 타입: {images.dtype}, 라벨 dtype: {labels.dtype}")
    break


=== Test Loader Info ===
총 배치 수 (len): 157
총 샘플 수: 10000
이미지 shape: torch.Size([64, 1, 28, 28])
레이블 shape: torch.Size([64])
데이터 타입: torch.float32, 라벨 dtype: torch.int64


In [ ]:
# PyTorch의 기본 신경망 모듈 nn.Module을 상속받아 DNN 클래스 정의
class DNN(nn.Module):

    # 클래스 초기화 메서드: 모델의 레이어들을 정의
    def __init__(self):
        # 부모 클래스(nn.Module)의 생성자를 호출해 초기화
        super(DNN, self).__init__()
        # 입력 데이터를 1차원으로 평탄화(28x28 → 784차원)
        self.flatten = nn.Flatten()
        # 여러 층을 순차적으로 쌓은 신경망 정의
        self.model = nn.Sequential(
            # 입력층(784) → 은닉층(128)
            nn.Linear(28*28, 128),
            # 활성화 함수 ReLU 적용
            nn.ReLU(),
            # 은닉층(128) → 은닉층(64)
            nn.Linear(128, 64),
            # 활성화 함수 ReLU 적용
            nn.ReLU(),
            # 은닉층(64) → 출력층(10 클래스)
            nn.Linear(64, 10)
        )

    # 순전파(forward) 과정 정의
    def forward(self, x):
        # 입력 데이터를 평탄화하여 벡터 형태로 변환
        x = self.flatten(x)
        # 정의한 모델 레이어들을 통과시켜 출력 반환
        return self.model(x)

#### 다음 코드 셀은 모델 학습에서의 경고 발생 제거

In [ ]:
# GPU의 SM(Streaming Multiprocessor) 수가 적어 경고 발생 가능 제거
import torch._inductor.config as inductor_cfg
# SM(Streaming Multiprocessor) = GPU의 핵심 연산 단위
inductor_cfg.max_autotune_gemm = False  # 해당 모드 사용 안 함

In [ ]:
# GPU(cuda)가 사용 가능하면 GPU에, 불가능하면 CPU에 연산을 수행하도록 장치를 설정함
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DNN 모델 객체를 생성하고, 앞서 설정한 장치(device)로 모델을 이동시킴
model = DNN().to(device)
# PyTorch 2.0의 torch.compile 기능을 활용해 모델을 컴파일하여 실행 속도를 최적화함
compiled_model = torch.compile(model)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(compiled_model.parameters(), lr=learning_rate)

In [ ]:
# 학습 루프
for epoch in range(epochs): # 전체 에폭 반복
    compiled_model.train() # 학습 모드 설정
    total_loss = 0
    correct = 0
    for data, target in train_loader: # 배치 반복
        data, target = data.to(device), target.to(device) # 입력/라벨 GPU로 이동

        # 순전파 → 손실 → 역전파 → 옵티마이저 스텝
        optimizer.zero_grad()
        output = compiled_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # 모델 예측 결과(output)에서 예측한 가장 큰 값의
        # 인덱스(argmax)를 뽑아 실제 정답(target)과 비교
        # 맞춘 개수를 세어서 correct 변수에 누적함
        correct += (output.argmax(1) == target).sum().item()

    # 정확도 계산
    accuracy = 100. * correct / len(train_loader.dataset)
    # 에폭별 결과 출력
    print(f"Epoch {epoch+1}, Loss: {total_loss:8.4f}, Accuracy: {accuracy:.2f}%")

Epoch 1, Loss: 322.5224, Accuracy: 90.30%
Epoch 2, Loss: 134.3002, Accuracy: 95.67%
Epoch 3, Loss:  91.6116, Accuracy: 97.11%
Epoch 4, Loss:  68.2185, Accuracy: 97.75%
Epoch 5, Loss:  53.2591, Accuracy: 98.26%


In [ ]:
# 테스트 루프
compiled_model.eval()
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = compiled_model(data)
        correct += (output.argmax(1) == target).sum().item()

test_accuracy = 100. * correct / len(test_loader.dataset)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 97.57%


### 종료